# Shader code

```c++
#version 300 es

// Swap pivoted entry parallel operation.

// per vertex input (not used, but at least one is required?)
in float row_input, col_input;

// The Gauss-Jordan expanded matrix in column major order.
uniform sampler2D expanded_matrix;

uniform int pivot_column;

uniform int swap_row;

uniform float epsilon;

out float pivot_value;  // feedback output

// retrieve row/col value from matrix in column major
float get_item(in int rownum, in int colnum) {
    // data is in R component only.
    return texelFetch(expanded_matrix, ivec2(rownum, colnum), 0).r;
}

void main() {
    // foil the optimizer -- use the dummy input so it isn't removed (that might cause errors downstream on Firefox)
    gl_Position = vec4(row_input, col_input, row_input, col_input);
    // Column major indexing
    int irow = gl_VertexID;
    int icol = gl_InstanceID;
    float pivot_entry = get_item(swap_row, pivot_column);
    pivot_value = 0.0;
    if (abs(pivot_entry) > epsilon) {
        // pivot entry is not too small.
        if (icol == pivot_column) {
            if (irow == pivot_column) {
                pivot_value = 1.0;  // pivot location
            }  
            // otherwise, pivot column, not at pivot location: 0.0
        } else {
            // not pivot column
            if (irow == pivot_column) {
                // pivot row, not pivot column
                float old_value = get_item(swap_row, icol);
                pivot_value = old_value / pivot_entry;
            } else {
                // not pivot row or pivot column
                // row swap logic
                if (irow == swap_row) {
                    irow = pivot_column;
                }
                float old_value = get_item(irow, icol);
                float pivot_column_value = get_item(swap_row, icol);
                float factor = pivot_column_value / pivot_entry;
                float pivot_row_value = get_item(irow, pivot_column);
                pivot_value = old_value - (factor * pivot_row_value);
            }
        }
    } else {
        // if pivot entry is too small, no change.
        pivot_value = get_item(irow, icol);
    }
}
```

In [1]:
import feedWebGL2.feedback as fd
import numpy as np
fd.widen_notebook()
np.set_printoptions(precision=4)

# https://stackoverflow.com/questions/20341614/numpy-array-row-major-and-column-major
def column_array(L):
    return np.array(L, dtype=np.float, order="F")

C = column_array([
    [1, 3, 5],
    [1, 2, 7]
])
C.ravel(order="K")

array([1., 1., 3., 2., 5., 7.])

In [2]:
initial_array = column_array([
    [-1, 1, 1],
    [ 1, 1,-1],
    [-1,-1,-1]
])

if 1:
    size = 100
    initial_array = (0.5 - np.random.random(size * size).reshape((size, size))) * size

def expanded_matrix(square_matrix):
    (n, m) = square_matrix.shape
    assert n == m, "matrix is not square: " + repr(square_matrix.shape)
    result = np.zeros( (n, n+n), dtype=np.float, order="F")
    result[:, :n] = square_matrix
    result[:, n:] = np.eye(n)
    return result

sq_expanded = expanded_matrix(initial_array)
expanded_list = sq_expanded.ravel(order="K")

In [3]:
shader_GLSL_code = """#version 300 es

    // Swap pivoted entry parallel operation.

    // per vertex input (not used, but at least one is required?)
    in float row_input, col_input;

    // The Gauss-Jordan expanded matrix in column major order.
    uniform sampler2D expanded_matrix;

    uniform int pivot_column;

    uniform int swap_row;
    
    uniform float epsilon;

    out float pivot_value;  // feedback output

    // retrieve row/col value from matrix in column major
    float get_item(in int rownum, in int colnum) {
        // data is in R component only.
        return texelFetch(expanded_matrix, ivec2(rownum, colnum), 0).r;
    }

    void main() {
        // foil the optimizer -- use the dummy input so it isn't removed (that might cause errors downstream on Firefox)
        gl_Position = vec4(row_input, col_input, row_input, col_input);
        // Column major indexing
        int irow = gl_VertexID;
        int icol = gl_InstanceID;
        float pivot_entry = get_item(swap_row, pivot_column);
        pivot_value = 0.0;
        if (abs(pivot_entry) > epsilon) {
            // pivot entry is not too small.
            if (icol == pivot_column) {
                if (irow == pivot_column) {
                    pivot_value = 1.0;  // pivot location
                }  
                // otherwise, pivot column, not at pivot location: 0.0
            } else {
                // not pivot column
                if (irow == pivot_column) {
                    // pivot row, not pivot column
                    float old_value = get_item(swap_row, icol);
                    pivot_value = old_value / pivot_entry;
                } else {
                    // not pivot row or pivot column
                    // row swap logic
                    if (irow == swap_row) {
                        irow = pivot_column;
                    }
                    float old_value = get_item(irow, icol);
                    float pivot_column_value = get_item(swap_row, icol);
                    float factor = pivot_column_value / pivot_entry;
                    float pivot_row_value = get_item(irow, pivot_column);
                    pivot_value = old_value - (factor * pivot_row_value);
                }
            }
        } else {
            // if pivot entry is too small, no change.
            pivot_value = get_item(irow, icol);
        }
    }
"""

(n, m) = sq_expanded.shape
sq_ravelled = list(sq_expanded.ravel(order="K"))

feedback_program = fd.FeedbackProgram(
    program = fd.Program(
        vertex_shader = shader_GLSL_code,
        feedbacks = fd.Feedbacks(
            pivot_value = fd.Feedback(num_components=1),
        ),
    ),
    runner = fd.Runner(
        vertices_per_instance = n,  # vertices == rows
        num_instances = m, # instances == columns
        uniforms = fd.Uniforms(
            pivot_column = fd.Uniform(default_value=[0], type="int"),
            swap_row = fd.Uniform(default_value=[0], type="int"),
            epsilon = fd.Uniform(default_value=[1e-15]),
        ),
        inputs = fd.Inputs(
            row_input = fd.Input(
                num_components = 1,
                from_buffer = fd.BufferLocation(
                    name = "row_buffer",
                ),
            ),
            col_input = fd.Input(
                num_components = 1,
                from_buffer = fd.BufferLocation(
                    name = "row_buffer",
                ),
            ),
        ),
        samplers = fd.Samplers(
            expanded_matrix = fd.Sampler(
                dim= "2D",
                from_texture= "matrix_texture",
            ),
        ),
    ),
    context = fd.Context(
        buffers = fd.Buffers(
            row_buffer = fd.Buffer(
                array = list(range(m)),
            ),
        ),
        textures= fd.Textures(
            matrix_texture = fd.Texture(
                height= m,
                width=  n,
                array= sq_ravelled,
            ),
        ),
    ),
)

def get_output():
    result = np.zeros(sq_expanded.shape, order="F")
    rresult = result.ravel(order="K")
    rresult[:] = feedback_program.element.get_output().sync_value()
    return result

# display the widget and debugging information
feedback_program.debugging_display()

In [4]:
#feedback_program.run()

feedback_program.js_init("""

    // allocate the output buffer (no initial value)
    element.output_buffer = element.feedback_context.buffer("output_buffer", 4);
    element.output_buffer.allocate_size( n * m );
    
    element.get_output_column = function(column_number) {
        // get column major column from the output buffer;
        var column_offset = n * column_number;
        return element.output_buffer.get_slice(column_offset, column_offset + n);
    };
    
    element.get_output = function() {
        debugger;
        var typed = element.feedback_runner.feedback_array("pivot_value");
        return Array.from(typed);
    };

    element.pivot_and_swap = function (swap_row, pivot_column) {
        // The main feedback loop operation is written in Javascript to reduce communications overhead.
        element.change_uniform_vector("swap_row", [swap_row]);
        element.change_uniform_vector("pivot_column", [pivot_column]);
        element.run_feedback_program();
        // afterward the run, link the feedback output on the GPU to the input texture
        var runner = element.feedback_runner;
        var context = element.feedback_context;
        var texture = context.textures.matrix_texture;
        runner.copy_feedback_to_buffer("pivot_value", "output_buffer");
        var from_buffer = context.buffers.output_buffer;
        texture.reload_from_buffer(from_buffer)
    };
    
    element.reduce_matrix = function(column0) {
        debugger;
        // perform Gauss-Jordan reduction on loaded matrix.
        // column0 is given as an argument because the output buffer not initialized on first iteration.
        for (var icol=0; icol<n; icol++) {
            var column = column0;
            if (icol > 0) {
                // get output column from previous iteration
                column = element.get_output_column(icol);
            }
            // determine swap row
            var swap_row = icol;
            var swap_max = 0.0;
            for (var i=icol; i<n; i++) {
                var value = Math.abs(column[i]);
                if (value > swap_max) {
                    swap_row = i;
                    swap_max = value;
                }
            }
            element.pivot_and_swap(swap_row, icol);
        }
    }
""", n=n, m=m)

In [5]:
#feedback_program.element.pivot_and_swap(1,2)
column0 = list(initial_array[:, 0])
feedback_program.element.reduce_matrix(column0)

['method', <class 'jp_proxy_widget.proxy_widget.CommandMaker'>::140704609221376, 'reduce_matrix', <class 'jp_proxy_widget.proxy_widget.LiteralMaker'>::140704609219304]

In [6]:
r = get_output()
r

array([[ 1.0000e+00,  0.0000e+00,  0.0000e+00, ...,  1.2179e-03,
        -1.0523e-03,  3.3701e-03],
       [ 0.0000e+00,  1.0000e+00,  0.0000e+00, ..., -5.5227e-03,
         2.6381e-03,  8.0591e-04],
       [ 0.0000e+00,  0.0000e+00,  1.0000e+00, ..., -5.0364e-04,
        -3.9753e-03,  9.5462e-03],
       ...,
       [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  6.5514e-04,
         4.3052e-03, -1.3893e-02],
       [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ..., -2.2182e-03,
         6.8243e-03, -4.4455e-03],
       [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  3.8172e-03,
        -1.7655e-03, -3.9127e-03]])

In [7]:
inverse = r[:,n:]
II = inverse.dot(initial_array)
II

array([[ 9.9998e-01, -6.2006e-08,  1.0776e-05, ...,  6.1112e-06,
         8.8182e-06,  2.9089e-05],
       [-1.4980e-05,  1.0000e+00,  3.7215e-06, ...,  5.5978e-06,
         8.8112e-06, -5.3155e-06],
       [ 2.8950e-06, -1.3127e-06,  1.0000e+00, ...,  8.2584e-06,
         3.1017e-05,  5.5285e-06],
       ...,
       [-9.6009e-06,  7.9215e-07,  7.1967e-06, ...,  9.9998e-01,
        -2.5902e-05,  2.4434e-05],
       [-5.5172e-06, -7.4697e-06,  3.8777e-06, ...,  5.3675e-06,
         1.0000e+00, -1.8220e-05],
       [ 1.4735e-05,  1.0871e-06, -1.5653e-06, ..., -1.1437e-05,
        -2.1879e-05,  9.9999e-01]])

In [8]:
np.allclose(np.eye(n), II, rtol=1e-03, atol=1e-03)

True

In [ ]:
np.linalg.det(II)